In [ ]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [ ]:
A1_Lab_1=pd.read_excel("A1_Lab-1.xlsx")
A1_Lab_2=pd.read_excel("A1_Lab-2.xlsx")
A2_Lab_1=pd.read_excel("A2_Lab-1.xlsx")
A2_Lab_2=pd.read_excel("A2_Lab-2.xlsx")

In [ ]:
well_A1=A1_Lab_1.merge(A1_Lab_2, how='outer', on='Depth',sort=True,suffixes=('', '_LAB-2')).assign(WELL='Well_A1')
depth_A1=well_A1['Depth']

In [ ]:
well_A2=A2_Lab_1.merge(A2_Lab_2, how='outer', on='Depth',sort=True, suffixes=('', '_LAB-2')).assign(WELL='Well_A2')
depth_A2=well_A2['Depth']

In [ ]:
set_AA=pd.concat([well_A1,well_A2])
set_AA.reset_index(drop=True, inplace=True)

In [ ]:
well_encoder = OrdinalEncoder(categories=[["Well_A1", "Well_A2"]])
well_encoder.fit(set_AA[["WELL"]])
set_AA["Well"] = well_encoder.transform(set_AA[["WELL"]])
set_AA=set_AA.drop(['Depth','WELL'], axis=1)

In [ ]:
No_NaN=set_AA.dropna()
No_NaN.reset_index(drop=True, inplace=True)
No_NaN.to_excel('set_AA-No_NaN.xlsx')

### Set_AA NaN figure

In [ ]:
set_AA_figure=set_AA.copy()
set_AA_figure.index += 1

set_AA_figure.columns=set_AA_figure.columns.str.replace('_LAB-2','')

plt.figure(figsize=(20, 15), dpi = 500)

colors = ["#6E9FD0", "black"]
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
plt.rcParams['font.family'] = 'arial'

sns.heatmap(set_AA_figure.isnull(), cbar=False, cmap=cmap, yticklabels=115)

plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14, rotation=0)

plt.show() 

### New NaN data (creating manually)

In [ ]:
df=pd.read_excel("New_NaN.xlsx").drop(['Depth'], axis=1)

In [ ]:
df["Well"] = well_encoder.transform(df[["WELL"]])
df=df.drop('WELL', axis=1)

### Set_AA' NaN figure

In [ ]:
df_figure=df.copy()
df_figure.index += 1
df_figure.columns=df_figure.columns.str.replace('_LAB-2','')

plt.figure(figsize=(20, 15), dpi = 500)

colors = ["#6E9FD0", "black"]
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
plt.rcParams['font.family'] = 'arial'

sns.heatmap(df_figure.isnull(), cbar=False, cmap=cmap, yticklabels=42, linewidth=.8)

plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14, rotation=0)

plt.show() 

## VALIDATION

### 1) Validation using LOOCV

In [ ]:
def LOOCV_scores(Imp_model,model_name):
    
    def NRMSE_Scorer(y_true, y_pred):
        rmse = mean_squared_error(
        y_true  = y_true,
        y_pred  = y_pred,
        squared = False
           )
    
        nrmse = rmse/(y.max()-y.min())    
    
        return nrmse

    nrmse_score = make_scorer(NRMSE_Scorer, greater_is_better = False)

    scores_1=pd.DataFrame()
    scores_2=pd.DataFrame()
    
    y_hats=Imp_model.loc[:,['Al2O3_LAB-2', 'SiO2_LAB-2', 'TiO2_LAB-2', 'Fe2O3_LAB-2', 'CaO_LAB-2', 'K2O_LAB-2']]

    for col in y_hats:
        X=Imp_model.iloc[:,0:29]
        y=y_hats[col]
        
        cv = LeaveOneOut()
        
        scoring= [nrmse_score, 'neg_mean_absolute_error']
    
        model = RandomForestRegressor(random_state=123, max_features=25, n_estimators= 150)
        
        for score in scoring:
            scores = cross_val_score(model, X, y, scoring=score, cv=cv)
            scores = absolute(scores)
            if score=='neg_mean_absolute_error':
                scores_1= scores_1.append({'Feature': col, f'MAE_{model_name}':mean(scores)}, ignore_index=True)
            else:
                scores_2= scores_2.append({f'NRMSE_{model_name}':mean(scores)}, ignore_index=True)
        

    Final_Scores=pd.concat([scores_1,scores_2], axis=1)       
        
    return Final_Scores


### 2) Validation of real versus predicted 

In [ ]:
def real_pred(Imp_model, model_name):
    nan=pd.isnull(df)
    nan= nan[nan.iloc[:,1] == True]
    indices=list(nan.index)
    
    real = No_NaN.loc[indices]    
    pred = Imp_model.loc[indices]
    
    error=pd.DataFrame()

    for i in real.columns[0:29]:
        
        rmse = mean_squared_error(
            y_true  = real[i],
            y_pred  = pred[i],
            squared = False
           )
                
        nrmse = rmse/(real[i].max()-real[i].min())     
       
        
        error= error.append({'Variable': i,f'RMSE_{model_name}': rmse.round(6), f'NRMSE_{model_name}': nrmse.round(6)},
                            ignore_index=True)
        
    
    return error          
    

### 3) Validation using a Grid Search

In [ ]:
def GS_scores(Imp_model,model_name):
    
    y_hats=Imp_model.loc[:,['Al2O3_LAB-2', 'SiO2_LAB-2', 'TiO2_LAB-2', 'Fe2O3_LAB-2', 'CaO_LAB-2', 'K2O_LAB-2']]
    y_hats.drop([49,53,76,155],axis=0, inplace=True)
    
    Imp_model.drop([49,53,76,155],axis=0,inplace=True)
    
    final_scores=pd.DataFrame()    
    
    for col in y_hats:
        X=Imp_model.iloc[:,0:29]
        y=y_hats[col]
        X_train, X_test, y_train, y_test = train_test_split(
                                            X,
                                            y,
                                            train_size   = 0.7,
                                            random_state = 123,
                                            shuffle      = True
                                        )
    
        # Evaluated Hyperparameters 
        param_grid = {'n_estimators': [30,70,150],
                      'max_features': [5, 10, 25],
                      'max_depth'   : [None, 3, 10, 20]
                         }

        # Grid    
        grid = GridSearchCV(
                estimator  = RandomForestRegressor(random_state = 123),
                param_grid = param_grid,
                scoring    = 'neg_root_mean_squared_error',
                n_jobs     = - 1,
                cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
                refit      = True,
                verbose    = 0,
                return_train_score = True
                )
        

        grid.fit(X = X_train, y = y_train)
      
        # Final model
            
        final_model = grid.best_estimator_
        predicted = final_model.predict(X = X_test)
    
        rmse = mean_squared_error(
            y_true  = y_test,
            y_pred  = predicted,
            squared = False
           )
            
        nrmse = rmse/(y_test.max()-y_test.min())    
       
    
        final_scores= final_scores.append({'Feature': col,f'RMSE_{model_name}': rmse.round(6),
                                           f'NRMSE_{model_name}': nrmse.round(6)},ignore_index=True)
    
    
    return final_scores
    

## IMPUTATION TESTS

## MICE with Linear Regression

### 1) Training

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [ ]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns = df.columns)
na_loc = df_scaled.isnull()
df_scaled[na_loc] = np.nan

In [ ]:
LR=LinearRegression()

Impute=IterativeImputer(estimator=LR,
                        missing_values=np.nan,
                        max_iter=5,
                        verbose=2,
                        imputation_order='roman',
                        random_state=0
                       )

ImputedData=Impute.fit_transform(df_scaled)

LR_IMP=pd.DataFrame(scaler.inverse_transform(ImputedData), columns=df.columns)

# Validation

LOOCV_scores(LR_IMP, 'LR')

real_pred(LR_IMP,'LR')

### 2) Imputation of the actual missing data

In [ ]:
set_AA_scaled=scaler.fit_transform(set_AA)
set_AA_scaled = pd.DataFrame(set_AA_scaled, columns = set_AA.columns)
na_loc = set_AA_scaled.isnull()
set_AA_scaled[na_loc] = np.nan

In [ ]:
Imputed_set_AA=Impute.transform(set_AA_scaled)
set_AA_LR_IMP=pd.DataFrame(scaler.inverse_transform(Imputed_set_AA), columns=set_AA.columns)

# Validation

GS_scores(set_AA_LR_IMP,'LR')

## Miceforest

### 1) Training

In [ ]:
import miceforest as mf

In [ ]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns = df.columns)
na_loc = df_scaled.isnull()
df_scaled[na_loc] = np.nan

In [ ]:
kernel=mf.ImputationKernel(df_scaled,
                           datasets=10,
                           save_all_iterations=True,
                           random_state=11
                          )

optimal_parameters, losses = kernel.tune_parameters(dataset=0,
                                                    optimization_steps=15,
                                                    random_state=11
                                                   )

kernel.mice(6, variable_parameters= optimal_parameters)

In [ ]:
unscaled_1=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(1)),columns=df.columns)
unscaled_2=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(2)),columns=df.columns)
unscaled_3=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(3)),columns=df.columns)
unscaled_4=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(4)),columns=df.columns)
unscaled_5=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(5)),columns=df.columns)
unscaled_6=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(6)),columns=df.columns)
unscaled_7=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(7)),columns=df.columns)
unscaled_8=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(8)),columns=df.columns)
unscaled_9=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(9)),columns=df.columns)
unscaled_0=pd.DataFrame(scaler.inverse_transform(kernel.complete_data(0)),columns=df.columns)

In [ ]:
MF_Imp=pd.concat([unscaled_0,unscaled_1,unscaled_2,unscaled_3,unscaled_4,unscaled_5,unscaled_6,unscaled_7,
                      unscaled_8,unscaled_9],).groupby(level=0).mean()

# Validtion

LOOCV_scores(MF_Imp,'MF')

real_pred(MF_Imp, 'MF')

### 2) Imputation of the actual missing data

In [ ]:
set_AA_scaled=scaler.fit_transform(set_AA)
set_AA_scaled = pd.DataFrame(set_AA_scaled, columns = set_AA.columns)
na_loc = set_AA_scaled.isnull()
set_AA_scaled[na_loc] = np.nan

In [ ]:
new_data_imputed = kernel.impute_new_data(new_data=set_AA_scaled)

In [ ]:
set_AA_MF_IMP = new_data_imputed.complete_data()

In [ ]:
set_AA_MF_IMP = pd.DataFrame(scaler.inverse_transform(set_AA_MF_IMP),columns=set_AA.columns)

# Validation

GS_scores(set_AA_MF_IMP,'MF')

## KNN

### 1) Training

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns = df.columns)
na_loc = df_scaled.isnull()
df_scaled[na_loc] = np.nan

In [ ]:
knn=KNNImputer(n_neighbors=4)

knn_=knn.fit_transform(df_scaled)

KNN_Imp=pd.DataFrame(scaler.inverse_transform(knn_), columns=df.columns)

# Validation

LOOCV_scores(KNN_Imp,'KNN')

real_pred(KNN_Imp,'KNN')

### 2) Imputation of the actual missing data

In [ ]:
set_AA_scaled=scaler.fit_transform(set_AA)
set_AA_scaled = pd.DataFrame(set_AA_scaled, columns = set_AA.columns)
na_loc = set_AA_scaled.isnull()
set_AA_scaled[na_loc] = np.nan

In [ ]:
set_AA_KNN_IMP=knn.transform(set_AA_scaled)

set_AA_KNN_IMP=pd.DataFrame(scaler.inverse_transform(set_AA_KNN_IMP), columns=set_AA.columns)

# Validation

GS_scores(set_AA_KNN_IMP,'KNN' )

## MIDAS

### 1) Training

In [ ]:
import tensorflow as tf
import MIDASpy as md

In [ ]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns = df.columns)
na_loc = df_scaled.isnull()
df_scaled[na_loc] = np.nan

In [ ]:
imputer = md.Midas(layer_structure = [256,256], vae_layer = False, seed = 89,input_drop = 0.75)
imputer.build_model(df_scaled)
imputer.train_model(training_epochs = 40)

In [ ]:
imputations = imputer.generate_samples(m=40).output_list 

In [ ]:
imputation_dfs = []
i=0
for imp in imputations:
    df_unscaled= scaler.inverse_transform(imputations[i])
    df_unscaled= pd.DataFrame(df_unscaled, columns = df_scaled.columns)
    imputation_dfs.append(df_unscaled)
    i=i+1

In [ ]:
n=1
for i in imputation_dfs:
    file_out = "midas_imp_" + str(n) + ".xlsx"
    i.to_excel(file_out, index=False)
    n += 1

In [ ]:
path="midas_imp_"
file_names= [path + str(i) + '.xlsx' for i in range(1, 41)]
data_all = pd.concat((pd.read_excel(i) for i in file_names), axis=1) # Import

In [ ]:
MIDAS_Imp=data_all.mean(axis=1, level=0)

# Validation

LOOCV_scores(MIDAS_Imp,'MIDAS')

real_pred(MIDAS_Imp,'MIDAS')

### 2) Imputation of the actual missing data

In [ ]:
set_AA_scaled=scaler.fit_transform(set_AA)
set_AA_scaled = pd.DataFrame(set_AA_scaled, columns = set_AA.columns)
na_loc = set_AA_scaled.isnull()
set_AA_scaled[na_loc] = np.nan

In [ ]:
imputer.build_model(set_AA_scaled)
imputer.train_model(training_epochs = 40)

In [ ]:
imputations = imputer.generate_samples(m=40).output_list 

In [ ]:
imputation_dfs = []
i=0
for imp in imputations:
    df_unscaled= scaler.inverse_transform(imputations[i])
    df_unscaled= pd.DataFrame(df_unscaled, columns = df_scaled.columns)
    imputation_dfs.append(df_unscaled)
    i=i+1

In [ ]:
n=1
for i in imputation_dfs:
    file_out = "midas_imp_" + str(n) + ".xlsx"
    i.to_excel(file_out, index=False)
    n += 1

In [ ]:
path="midas_imp_"
file_names= [path + str(i) + '.xlsx' for i in range(1, 41)]
data_all = pd.concat((pd.read_excel(i) for i in file_names), axis=1) # Import
set_AA_MIDAS_IMP=data_all.mean(axis=1, level=0)

# Validation

GS_scores(set_AA_MIDAS_IMP,'MIDAS')